In [1]:
# multivariate lstm example
from numpy import array
from numpy import hstack
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
# from keras.layers import LSTM, Dense
from sklearn.preprocessing import StandardScaler
import numpy as np
import pandas as pd

2023-10-28 14:28:48.942318: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-10-28 14:28:48.943796: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-10-28 14:28:48.962842: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-10-28 14:28:48.962869: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-10-28 14:28:48.962884: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to regi

In [1]:
from empresa4.datasets import get_nombres_datasets, get_dataset
get_nombres_datasets()

df = get_dataset('01_productos_todos')
df

FileNotFoundError: [Errno 2] No such file or directory: './datasets/tb_sellout_01_productos_todos.csv'

In [2]:
# split a multivariate sequence into samples
def split_sequences(sequences, n_steps):
	X, y = list(), list()
	for i in range(len(sequences)):
		# find the end of this pattern
		end_ix = i + n_steps
		# check if we are beyond the dataset
		if end_ix > len(sequences):
			break
		# gather input and output parts of the pattern
		seq_x, seq_y = sequences[i:end_ix, :-1], sequences[end_ix-1, -1]
		X.append(seq_x)
		y.append(seq_y)
	return array(X), array(y)

In [5]:
def make_dataset():
	# Load your data
    df = pd.read_csv("../Datasets Clase 2/tb_sellout_01_todos.csv")

    # Sort and filter data
    df = df.sort_values(['product_id', 'periodo'])
    df = df[df['periodo'] <= 201904]  # Assuming you have data up to 201904

    # Create lagged features
    df['lag_cust_request_qty'] = df.groupby('product_id')['cust_request_qty'].shift(2)
    df['lag_cust_request_tn'] = df.groupby('product_id')['cust_request_tn'].shift(2)
    df['lag_tn'] = df.groupby('product_id')['tn'].shift(2)

    # Drop NaNs created due to lagging
    df = df.dropna()
    df = df[["product_id", "periodo", "lag_cust_request_qty","lag_cust_request_tn", "lag_tn"]]
    return df


In [3]:
def get_input_for_predictions(df, n_steps):
    max_periodo = df['periodo'].max()
    min_periodo = max_periodo - (n_steps + 1)
    filtered_df = df[df['periodo'] >= min_periodo]
    return filtered_df


In [6]:
# horizontally stack columns
dataset = make_dataset()
dataset

,product_id,periodo,lag_cust_request_qty,lag_cust_request_tn,lag_tn
2466,20001,201703,479.0,937.72717,934.77222
3699,20001,201704,432.0,833.72187,798.01620
4932,20001,201705,509.0,1330.74697,1303.35771
6165,20001,201706,279.0,1132.94430,1069.96130
7398,20001,201707,701.0,1550.68936,1502.20132
...,...,...,...,...,...
29235,21299,201812,0.0,0.00000,0.00000
30468,21299,201901,0.0,0.00000,0.00000
31701,21299,201902,0.0,0.00000,0.00000
32934,21299,201903,0.0,0.00000,0.00000


In [7]:
# define input sequence
in_seq1 = array(dataset['lag_cust_request_qty'])
in_seq2 = array(dataset['lag_cust_request_tn'])
out_seq = array(dataset['lag_tn'])
# convert to [rows, columns] structure
in_seq1 = in_seq1.reshape((len(in_seq1), 1))
in_seq2 = in_seq2.reshape((len(in_seq2), 1))
out_seq = out_seq.reshape((len(out_seq), 1))
# horizontally stack columns
dataset = hstack((in_seq1, in_seq2, out_seq))

In [8]:
dataset

array([[ 479.     ,  937.72717,  934.77222],
       [ 432.     ,  833.72187,  798.0162 ],
       [ 509.     , 1330.74697, 1303.35771],
       ...,
       [   0.     ,    0.     ,    0.     ],
       [   0.     ,    0.     ,    0.     ],
       [   0.     ,    0.     ,    0.     ]])

In [10]:
# choose a number of time steps
n_steps = 2
# convert into input/output
X, y = split_sequences(dataset, n_steps)

In [ ]:

# the dataset knows the number of features, e.g. 2
n_features = X.shape[2]
# define model
model = Sequential()
model.add(LSTM(50, activation='relu', input_shape=(n_steps, n_features)))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')
# fit model
model.fit(X, y, epochs=200, verbose=0)
# demonstrate prediction
x_input = array([[80, 85], [90, 95], [100, 105],[110, 115],[120, 125]])
x_input = x_input.reshape((1, n_steps, n_features))
yhat = model.predict(x_input, verbose=0)
print(yhat)